## inspector tool

In [1]:
!pip install ffmpeg-python

In [2]:
import ee
import geemap

In [3]:
ee.Authenticate()
ee.Initialize()

## 토지 변화 - 새만금

**Landsat 7**
- [Landsat 7 Collection 1 Tier 1 8-Day TOA Reflectance Composite](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LE07_C01_T1_8DAY_TOA)
- 1999-01-01T00:00:00Z–2021-12-27T00:00:00Z

**새만금**
- 1991년 사업이 시작되어, 2010년 4월 27일 새만금 방조제가 완공
- 사업기간 : 방조제(1991-2010년)/내부개발(2010-2020년)

### 새만금 Split map

In [4]:
saemangeum_roi = ee.Geometry.BBox(126.36, 35.70, 126.89, 35.98)

In [5]:
def create_split(start_year, end_year):
  left_img = ee.ImageCollection('LANDSAT/LE07/C01/T1_TOA').filterBounds(saemangeum_roi).filterDate(f'{start_year}-01-01', f'{start_year}-12-31').median()
  right_img = ee.ImageCollection('LANDSAT/LE07/C01/T1_TOA').filterBounds(saemangeum_roi).filterDate(f'{end_year}-01-01', f'{end_year}-12-01').median()

  vis = {'bands': ['B3', 'B2', 'B1'], 'min': 0, 'max': 0.3}

  left_layer = geemap.ee_tile_layer(left_img, vis, f"Landsat {start_year}")
  right_layer = geemap.ee_tile_layer(right_img,  vis, f"Landsat {end_year}")

  return left_layer, right_layer

In [6]:
left2000, right2011 = create_split(2000, 2011)

Map = geemap.Map()
Map.centerObject(saemangeum_roi,11)
Map.split_map(left2000, right2011)
Map

Map(center=[35.840014603466535, 126.62499999999989], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [7]:
left2010, right2011 = create_split(2010, 2021)

Map = geemap.Map()
Map.centerObject(saemangeum_roi,11)
Map.split_map(left2010, right2011)
Map

Map(center=[35.840014603466535, 126.62499999999989], controls=(ZoomControl(options=['position', 'zoom_in_text'…

### Landsat 영상을 사용해 2000~2023년 새만금 변화 타임랩스 GIF 생성

In [8]:
timelapse = geemap.landsat_timelapse(
    saemangeum_roi,
    out_gif="Saemangeum.gif",
    start_year=2000,
    end_year=2023,
    start_date="06-01",
    end_date="09-01",
    bands=["SWIR1", "NIR", "Red"],
    frames_per_second=10,
    title="Landsat Timelapse",
    progress_bar_color="blue",
    mp4=True,
)
geemap.show_image(timelapse)

Generating URL...
Please wait ...
The GIF image has been saved to: /content/Saemangeum.gif


Output()

## 사계절 식생

**MODIS**
- [MOD13Q1.061 Terra Vegetation Indices 16-Day Global 250m](https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MOD13Q1)
- 2000-02-18T00:00:00Z–2024-03-21T00:00:00Z
- bands: NDVI, EVI, DetailedQA

In [9]:
def create_img(month):
  return ee.ImageCollection("MODIS/061/MOD13Q1").filterDate(f'2023-{month}-01', f'2023-{month}-28').select('NDVI')

imgs = [create_img('04'), create_img('07'), create_img('10'), create_img('12')]

ndviVis = {
  'min': 0,
  'max': 8000,
  'palette': [
    'ffffff', 'ce7e45', 'df923d', 'f1b555', 'fcd163', '99b718', '74a901',
    '66a000', '529400', '3e8601', '207401', '056201', '004c00', '023b01',
    '012e01', '011d01', '011301'
  ],
}

vis_params = [ndviVis]

labels = [
    "spring(4) NDVI",
    "summer(7) NDVI",
    "fall(10) NDVI",
    "winter(12) NDVI"
]

geemap.linked_maps(
    rows=2,
    cols=2,
    height="400px",
    center=[37.53, 128.88],
    zoom=6,
    ee_objects=imgs,
    vis_params=vis_params,
    labels=labels,
    label_position="topright",
)


GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

## 기후 변화

**TerraClimate**
- [TerraClimate: Monthly Climate and Climatic Water Balance for Global Terrestrial Surfaces, University of Idaho](https://developers.google.com/earth-engine/datasets/catalog/IDAHO_EPSCOR_TERRACLIMATE)
- 1958-01-01T00:00:00Z–2023-12-01T00:00:00Z
- bands: pr, tmmx, vpd

In [10]:
weather = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE")

In [11]:
weather = ee.ImageCollection([ee.Image(f"IDAHO_EPSCOR/TERRACLIMATE/{year}08").select('tmmx') for year in range(2005, 2024)])
weather_layer_names = ["weather " + str(year) for year in range(2005, 2024)]

In [12]:
maximum_temperature_vis = {
    'min': 200.0,
    'max': 300.0,
    'palette': [
    '1a3678', '2955bc', '5699ff', '8dbae9', 'acd1ff', 'caebff', 'e5f9ff',
    'fdffb4', 'ffe6a2', 'ffc969', 'ffa12d', 'ff7c1f', 'ca531a', 'ff0000',
    'ab0000'
  ],}

### Split Map

In [13]:
m = geemap.Map()
m.ts_inspector(
    left_ts=weather,
    right_ts=weather,
    left_names=weather_layer_names,
    right_names=weather_layer_names,
    left_vis=maximum_temperature_vis,
    right_vis=maximum_temperature_vis,
)
m.centerObject(saemangeum_roi, zoom=6)
m

Map(center=[35.840014603466535, 126.62499999999989], controls=(WidgetControl(options=['position', 'transparent…

### Slider

In [14]:
ms = geemap.Map()
ms.centerObject(saemangeum_roi, zoom=6)
ms.add_time_slider(weather, maximum_temperature_vis, labels=weather_layer_names, time_interval=5, opacity=0.8)
ms

Map(center=[35.840014603466535, 126.62499999999989], controls=(WidgetControl(options=['position', 'transparent…